In [6]:
import hail as hl
hl.init()

Running on Apache Spark version 2.4.1
SparkUI available at http://nucleus.cels.anl.gov:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.28-61941242c15d
LOGGING: writing to /vol/bmd/yanyul/GitHub/ptrs-ukb/notebook/hail-20191224-0849-0.2.28-61941242c15d.log


In [4]:
import pandas as pd

In [3]:
gwas_sample = hl.import_table(
    '/vol/bmd/yanyul/UKB/gwas_on_subset/gwas_runs_in_tsv/gwas_in_tsv_subset13_x_height.tsv',
    key = ['rsid', 'variant']
)
f = pd.read_csv('../external_data/martin_et_al_2019ng_table_s6_trait_description.tsv', sep = '\t')
traits = [ i.lower() for i in f['short'].to_list() ]
clumped_snp_files = []
for i in traits:
    for j in range(1, 18):
        clumped_snp_files.append('/vol/bmd/yanyul/UKB/ld_clump/gwas_clump_x_subset' + str(j) + '_x_' + i + '.clumped_snp')
ht_var = hl.import_table(clumped_snp_files, no_header = True, key = 'f0')
ht_var = ht_var.key_by('f0')
gwas_sample = gwas_sample.annotate(is_clump_var = ht_var[gwas_sample.rsid])
ht_var = gwas_sample.filter(hl.is_defined(gwas_sample.is_clump_var))
chr_all = '{' + ','.join([str(i) for i in range(22, 23)]) + '}'
index_dic = {
    f'/vol/bmd/data/ukbiobank/genotypes/v3/ukb_imp_chr{i}_v3.bgen' : f'/vol/bmd/yanyul/UKB/bgen_idx/ukb_imp_chr{i}_v3.bgen.idx2' for i in range(1, 23)
}
mt = hl.import_bgen(
    '/vol/bmd/data/ukbiobank/genotypes/v3/ukb_imp_chr' + chr_all + '_v3.bgen', 
    index_file_map = index_dic,
    entry_fields = ['dosage'],
    sample_file = '/vol/bmd/data/ukbiobank/genotypes/v3/ukb19526_imp_chr1_v3_s487395.sample',
    variants = hl.parse_variant(ht_var.variant)
)

2019-12-23 16:29:43 Hail: INFO: Reading table with no type imputation
  Loading column 'variant' as type 'str' (type not specified)
  Loading column 'minor_allele' as type 'str' (type not specified)
  Loading column 'minor_AF' as type 'str' (type not specified)
  Loading column 'low_confidence_variant' as type 'str' (type not specified)
  Loading column 'n_complete_samples' as type 'str' (type not specified)
  Loading column 'AC' as type 'str' (type not specified)
  Loading column 'ytx' as type 'str' (type not specified)
  Loading column 'beta' as type 'str' (type not specified)
  Loading column 'se' as type 'str' (type not specified)
  Loading column 'tstat' as type 'str' (type not specified)
  Loading column 'pval' as type 'str' (type not specified)
  Loading column 'rsid' as type 'str' (type not specified)
  Loading column 'ref' as type 'str' (type not specified)
  Loading column 'alt' as type 'str' (type not specified)

2019-12-23 16:29:44 Hail: INFO: Reading table with no type imp

In [4]:
# mt.count_rows()

In [2]:
indiv_files = []
for i in ['African', 'Chinese', 'Indian']:
    indiv_files.append('/vol/bmd/yanyul/GitHub/ptrs-ukb/output/data_split/' + i + '.txt')

# for subset i
# for trait height
i = 1
trait = 'height'

for prefix in [ 'British-test-', 'British-validation-' ]:
    indiv_files.append('/vol/bmd/yanyul/GitHub/ptrs-ukb/output/data_split/' + prefix + str(i) + '.txt')
    indiv_files.append('/vol/bmd/yanyul/GitHub/ptrs-ukb/output/data_split/' + prefix + str(i) + '.txt')

clump_file = '/vol/bmd/yanyul/UKB/ld_clump/gwas_clump_x_subset' + str(i) + '_x_' + trait + '.clumped_snp'
gwas_file = '/vol/bmd/yanyul/UKB/gwas_on_subset/gwas_runs_in_tsv/gwas_in_tsv_subset' + str(i) + '_x_' + trait + '.tsv'

In [6]:
ht_indiv = hl.import_table(indiv_files, key = ['f0'], no_header = True, delimiter = ' ')
gwas_tsv = hl.import_table(gwas_file, key = ['rsid'], types = {'beta' : hl.tfloat, 'se' : hl.tfloat, 'pval' : hl.tfloat})
clump_snp = hl.import_table(clump_file, key = ['f0'], no_header = True)
# gwas_tsv = gwas_tsv.annotate(is_clump = clump_snp[gwas_tsv.rsid])
gwas_tsv = gwas_tsv.filter(hl.is_defined(clump_snp[gwas_tsv.rsid]))

2019-12-23 16:40:21 Hail: INFO: Reading table with no type imputation
  Loading column 'f0' as type 'str' (type not specified)
  Loading column 'f1' as type 'str' (type not specified)

2019-12-23 16:40:21 Hail: INFO: Reading table with no type imputation
  Loading column 'variant' as type 'str' (type not specified)
  Loading column 'minor_allele' as type 'str' (type not specified)
  Loading column 'minor_AF' as type 'str' (type not specified)
  Loading column 'low_confidence_variant' as type 'str' (type not specified)
  Loading column 'n_complete_samples' as type 'str' (type not specified)
  Loading column 'AC' as type 'str' (type not specified)
  Loading column 'ytx' as type 'str' (type not specified)
  Loading column 'beta' as type 'float64' (user-specified)
  Loading column 'se' as type 'float64' (user-specified)
  Loading column 'tstat' as type 'str' (type not specified)
  Loading column 'pval' as type 'float64' (user-specified)
  Loading column 'rsid' as type 'str' (type not speci

In [7]:
k = hl.parse_variant(gwas_tsv.variant)
gwas_tsv = gwas_tsv.annotate(**k)

In [8]:
gwas_tsv = gwas_tsv.key_by(gwas_tsv.locus, gwas_tsv.alleles)
# gwas_tsv = gwas_tsv.repartition(gwas_tsv.n_partitions())

In [9]:
gwas_tsv = gwas_tsv.repartition(40)
gwas_tsv = gwas_tsv.cache()

2019-12-23 16:40:25 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-12-23 16:40:26 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-12-23 16:40:52 Hail: INFO: Ordering unsorted dataset with network shuffle


In [10]:
mt_this = mt.filter_rows(hl.is_defined(gwas_tsv[mt.locus, mt.alleles]))
mt_this = mt_this.filter_cols(hl.is_defined(ht_indiv[mt_this.s]))

In [11]:
mt_this = mt_this.repartition(100)
mt_this = mt_this.cache()

In [12]:
mt_this.count()

(29543, 18950)

In [ ]:
# gwas_annot = 

In [17]:
mt_this = mt_this.annotate_rows(
    gwas_beta = gwas_tsv[mt_this.locus, mt_this.alleles].beta, 
    gwas_pval = gwas_tsv[mt_this.locus, mt_this.alleles].pval
)

In [18]:
pval_thresholds = [ 5e-8, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 0.05, 0.1, 0.5, 1 ]

In [19]:
prs = {
    'pval_thres_' + str(i) : hl.agg.sum(mt_this.gwas_beta * mt_this.dosage * hl.int(mt_this.gwas_pval < i)) for i in pval_thresholds
}

In [20]:
mt_this = mt_this.annotate_cols(**prs)

In [21]:
mt_this = mt_this.cache()

In [26]:
mt_this['pval_thres_5e-08'].show()

s,pval_thres_5e-08
str,float64
"""5595764""",5.88e-01
"""5172041""",3.82e-01
"""2266650""",-2.10e-01
"""4860485""",6.71e-01
"""3131191""",1.20e+00
"""1528789""",7.58e-03
"""3197109""",3.44e-03
"""5008884""",2.61e-01
"""1517276""",8.51e-01


In [3]:
f_clump = open(clump_file, 'r')
l_clump = f_clump.readlines()
f_clump.close()
dic_clump = {}
for i in l_clump:
    dic_clump[i.strip()] = 1
out = []
with open(gwas_file, 'r') as f:
    header = next(f)
    out.append(header)
    rsid_idx = header.split('\t').index('rsid')
    for i in f:
        k = i.split('\t')[rsid_idx]
        if k in dic_clump:
            out.append(i)
            

In [ ]:
# d = pd.read_csv('\n'.join(out))


KeyboardInterrupt



In [5]:
len(out)

2007626

In [7]:
f = hl.read_matrix_table('/vol/bmd/yanyul/UKB/tmp/test_prs_x_test_subset1_x_wbc.prs.ht')

In [16]:
f.col.export('test.tsv.bgz')

2019-12-24 08:59:32 Hail: INFO: merging 2 files totalling 869.1K...
2019-12-24 08:59:32 Hail: INFO: while writing:
    test.tsv.bgz
  merge time: 29.500ms


In [19]:
!zcat test.tsv.bgz|head

s	pval_thres_5e-08	pval_thres_1e-07	pval_thres_1e-06	pval_thres_1e-05	pval_thres_0.0001	pval_thres_0.001	pval_thres_0.01	pval_thres_0.1	pval_thres_0.05	pval_thres_0.5	pval_thres_1.0
5595764	9.6724e-02	9.6724e-02	3.0077e-02	7.6352e-03	2.0318e-01	3.6196e-01	5.2011e-01	2.2488e-01	-7.5449e-01	5.0274e-01	5.9248e-01
5172041	-5.7805e-04	-5.7805e-04	7.5673e-02	9.2538e-02	2.6782e-01	6.0770e-01	5.2760e-01	2.8239e-01	-7.1257e-01	5.8479e-01	5.0037e-01
2266650	-1.4736e-03	-1.4736e-03	-5.6721e-02	-8.9969e-03	4.4957e-02	3.3760e-01	2.1838e-01	3.6890e-01	-5.9017e-01	4.7468e-01	3.4717e-01
4860485	2.4917e-02	2.4917e-02	-1.5172e-02	-9.5946e-03	9.8818e-02	5.4127e-01	6.9360e-01	-3.0089e-01	-8.6135e-01	2.9628e-01	2.8129e-01
3131191	3.3175e-02	3.3175e-02	6.0356e-02	1.1818e-02	-9.2583e-03	1.4733e-01	1.4158e-01	-7.9820e-01	-1.5161e+00	-6.9655e-01	-8.3469e-01
1528789	2.3629e-02	2.3629e-02	4.8931e-02	-6.1449e-02	1.0919e-01	4.8954e-01	1.4597e+00	3.2050e+00	1.8402e+00	4.1675e+00	4.0017e+00
3197109	-4.3886e-02	-4.38